In [1]:
import get_started_doing_something_variables
import get_stopped_doing_something_variables
import get_all_vars

import pandas as pd
import read_a_combination_of_variables
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import settings
import ast

bucket = S3BucketUtils()

def get_key_based_on_value_in_a_dict(dict_, value_of_interest):
    for key, value in dict_.items():
        if value == value_of_interest:
            return key

def get_model_number(var_, model_names):
    if len(model_names[model_names['model_name']==var_]['model_number'])>0:
        model_number = model_names[model_names['model_name']==var_]['model_number'].iloc[0]
    else:
        multiple_vars = model_names[(model_names['model_name'].apply(lambda x: ',' in x))]
        for i in range(0, len(multiple_vars)):
            if var_ in multiple_vars['model_name'].iloc[i]:
                model_number = multiple_vars['model_number'].iloc[i]
                break
    return model_number

def get_pairs_of_variables(churn_based_on_behaviour_dir, date_dir, model_type):
    vars_periods_to_look_at_thresholds_to_use = \
    bucket.load_csv_from_s3(file_name = churn_based_on_behaviour_dir + 'data/' + date_dir +\
                     '/vars_periods_to_look_at_thresholds_to_use.csv')

    period_to_look_at =\
    vars_periods_to_look_at_thresholds_to_use[['variable_base_name', 'period_to_look_at', 'number_of_months']].\
    apply(lambda x: (x['variable_base_name'], x['period_to_look_at']), axis=1).tolist()


    (all_vars, all_vars_base_names) = get_all_vars.main()
    first_var = dict.fromkeys(all_vars_base_names.values())
    second_var = dict.fromkeys(all_vars_base_names.values())

    if model_type == 'started_doing_something':
        for i in range(0, len(period_to_look_at)):
            first_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_'+period_to_look_at[i][1]
            second_var[period_to_look_at[i][0]] = \
            'had_'+period_to_look_at[i][0]+'_before_and_didnt_'+period_to_look_at[i][1]
    elif model_type == 'stopped_doing_something':
        for i in range(0, len(period_to_look_at)):
            first_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_before'
            second_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_'+period_to_look_at[i][1]

    return (first_var, second_var)

date_of_analysis = input('date of analysis: ')
date_dir = date_of_analysis.replace('-', '_')

old_models_path = 'combinations_and_names_Dec_10th/'
curr_models_path = 'combinations_and_names_Dec_21st/'

def main(model_type):
    vars_diff = \
    pd.read_csv('data/'+date_dir+'/exports/'+model_type+'/not_among_newest_important_variables.csv')
    variables_not_in_the_current_combination = dict.fromkeys(vars_diff['variable'])
    (first_var, second_var) = \
    get_pairs_of_variables(churn_based_on_behaviour_dir='churn_analysis_based_on_behaviour/', date_dir=date_dir,\
                           model_type=model_type)
    for var_ in vars_diff['variable'].unique():
#         print('Var name: '+var_)
        if var_ in first_var.values():
            base_var_ = get_key_based_on_value_in_a_dict(dict_=first_var, value_of_interest=var_)
        elif var_ in second_var.values():
            base_var_ = get_key_based_on_value_in_a_dict(dict_=second_var, value_of_interest=var_)
        else:
            base_var_ = var_

        old_model_names = \
        pd.read_csv('combinations_of_variables_that_are_not_dependent/'+old_models_path+'model_names.csv')
        old_combinations = \
        pd.read_csv('combinations_of_variables_that_are_not_dependent/'+old_models_path+'combinations_of_variables.csv')
        models_with_p_below_0_2 = []
        for model_num in old_model_names['model_number'][1:]:
            files = \
            os.listdir('data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/models_with_old_combinations/model_'+str(model_num)+'/')

            for i in range(0, len(files)):
                old_coefs_and_pvalues = \
                pd.read_csv('data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/models_with_old_combinations/model_'+str(model_num)+'/'+\
                   files[i])
                if len(old_coefs_and_pvalues[(old_coefs_and_pvalues['covariate']==var_)&\
                             (old_coefs_and_pvalues['p']<0.2)]):
                    models_with_p_below_0_2.append(model_num)
                    break

        curr_model_names =\
        pd.read_csv('combinations_of_variables_that_are_not_dependent/'+curr_models_path+'/model_names.csv')
        model_num = \
        get_model_number(var_=base_var_, model_names=curr_model_names)
        curr_combinations = \
        pd.read_csv('combinations_of_variables_that_are_not_dependent/'+curr_models_path+'/combinations_of_variables.csv')

        not_in_the_old_combination_lists = []
        for i in range(0, len(models_with_p_below_0_2)):
            not_in_the_old_combination = \
            [x for x in curr_combinations['model_'+str(model_num)].unique() if x not in old_combinations['model_'+str(models_with_p_below_0_2[i])].unique()\
            and x==x]
            not_in_the_old_combination_lists.append(not_in_the_old_combination)

        variables_not_in_the_current_combination[var_] = [ast.literal_eval(x) for x in list(dict.fromkeys([str(sorted(x)) for x in not_in_the_old_combination_lists]))]
        
    for_all_vars = []
    for key in variables_not_in_the_current_combination.keys():
        all_vars_in_all_combinations = []
        for i in range(0, len(variables_not_in_the_current_combination[key])):
            all_vars_in_all_combinations.extend(variables_not_in_the_current_combination[key][i])
        all_vars_in_all_combinations = list(dict.fromkeys(all_vars_in_all_combinations))

        combinations_dfs = \
        pd.DataFrame({'main_variable':[key]*len(all_vars_in_all_combinations), 'vars_not_in_the_current_model':all_vars_in_all_combinations})

        first_tmp_df = \
        pd.DataFrame({'vars_not_in_the_current_model':variables_not_in_the_current_combination[key][0],\
                 'combination_'+str(1):variables_not_in_the_current_combination[key][0]})

        combinations_dfs = \
        combinations_dfs.merge(first_tmp_df, on='vars_not_in_the_current_model',\
                          how = 'left')

        for i in range(1, len(variables_not_in_the_current_combination[key])):
            tmp_df = \
        pd.DataFrame({'vars_not_in_the_current_model':variables_not_in_the_current_combination[key][i],\
                     'combination_'+str(i+1):variables_not_in_the_current_combination[key][i]})
            combinations_dfs = \
        combinations_dfs.merge(tmp_df, on='vars_not_in_the_current_model',\
                              how = 'left')

        for_all_vars.append(combinations_dfs)
    final_export = for_all_vars[0]
    for i in range(1, len(for_all_vars)):
        final_export = pd.concat([final_export, for_all_vars[i]], sort=False)
        
    final_export.\
    to_csv('data/'+date_dir+'/exports/'+model_type+'/differences_in_combinations_of_variables.csv',\
          index = False)

date of analysis: 2021-12-01


In [2]:
for model_type in ['started_doing_something', 'stopped_doing_something']:
    main(model_type=model_type)